In [12]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from trialblazer.models import (
    get_morgan2,
    compute_2Drdkit,
    MLP_simulation_test,
    MLP_cv,
    MLP_decision_threshold_optimization,
    RF_cv,
    feature_selection
)
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

In [13]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
Training_data = pd.read_csv("../Data/Training_data_withoutInfo.csv", sep='|')
Target_features = pd.read_csv('../Data/training_target_features.csv')

# Descriptors

## Predicted bioactivity fingerprints (PBFPs)

In [15]:
active_targets_merged_forlist = Target_features.drop(columns=['SmilesForDropDu'])
targets_list = list(active_targets_merged_forlist.columns)
print(len(targets_list))

In [16]:
Training_data = Training_data.merge(Target_features, how='left',on='SmilesForDropDu')
Training_data[targets_list].isna().any().sum()

## Morgan2 fingerprints (M2FPs)

In [17]:
Training_data.Molecule = Training_data.SmilesForDropDu.apply(Chem.MolFromSmiles)
Training_data['fp'] = Training_data.Molecule.apply(get_morgan2)
morgan2_cols = ['morgan2_b'+ str(i) for i in list(range(2048))]
Training_data[morgan2_cols] = Training_data['fp'].to_list()

## Physicochemical descriptors (PCD)

If you are not using PCD, don't need to run this section, becuase it will remove one datapoint from Trianing_data

In [18]:
descriptors_list = [x[0] for x in Descriptors._descList]
print(len(descriptors_list))

In [19]:
Training_data.Molecule = Training_data.SmilesForDropDu.apply(Chem.MolFromSmiles)
Training_data['desc'] = Training_data.Molecule.apply(compute_2Drdkit)

In [20]:
calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
desc_cols = list(calc.GetDescriptorNames())
Training_data[desc_cols] = Training_data.desc.to_list()
Training_data.drop("desc",axis=1,inplace=True)

In [ ]:
Training_data[desc_cols].dropna(inplace=True)
Training_data[desc_cols].isnull().values.any()

In [ ]:
Training_data[desc_cols][Training_data[desc_cols].isna().any(axis=1)] # somehow dropna didn't remove this.

In [ ]:
Training_data.iloc[1272,:]

In [ ]:
Training_data.drop(labels=1272, axis=0,inplace=True)
Training_data.reset_index(inplace=True)

# 10-fold cross validation

## model based on 208 physicochemical descriptors

In [ ]:
X = Training_data[desc_cols]
y = Training_data.Mark

In [ ]:
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)

MLP

In [ ]:
MLP_cv(X_scaled,y)

## model based on M2FPs

In [28]:
X = Training_data[morgan2_cols]
y = Training_data.Mark

MLP

In [ ]:
opt_threshold_ap, opt_threshold_mean = MLP_decision_threshold_optimization(
    X,
    y,
    opt_num_feature=750
    ) 

In [30]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from scipy.stats import bernoulli
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.neural_network import MLPClassifier
from tqdm import tqdm
def MLP_cv(X, y, opt_num_feature="all", threshold=None) -> None:
    selector = SelectKBest(f_classif, k=opt_num_feature)
    X_new = selector.fit_transform(X, y)
    cv = StratifiedKFold(n_splits=10)
    if opt_num_feature == "all" and threshold is None:
        classifier = MLPClassifier(random_state=42)
    else:
        classifier = MLPClassifier(
            hidden_layer_sizes=(70,),
            random_state=42,
            learning_rate_init=0.0001,
            max_iter=200,
        )
    tprs, aucs, MCCs, cms, baccs, recalls, precisions = (
        [],
        [],
        [],
        [],
        [],
        [],
        [],
    )
    mean_fpr = np.linspace(0, 1, 100)
    fig, ax = plt.subplots()

    for i, (train, test) in enumerate(cv.split(X_new, y)):
        classifier.fit(X_new[train], y[train])

        y_prob = classifier.predict_proba(X_new[test])[:, 1]

        if threshold is None:  
            y_pred_opt = classifier.predict(X_new[test])
        else:  
            y_pred_opt = (y_prob >= threshold).astype(int)

        mcc = matthews_corrcoef(y[test], y_pred_opt)
        bacc = balanced_accuracy_score(y[test], y_pred_opt)
        cm = confusion_matrix(y[test], y_pred_opt, labels=classifier.classes_)
        rec = recall_score(y[test], y_pred_opt)
        prec = precision_score(y[test], y_pred_opt)

        MCCs.append(mcc)
        baccs.append(bacc)
        cms.append(cm)
        recalls.append(rec)
        precisions.append(prec)
        
        fpr, tpr, _ = roc_curve(y[test], y_prob)
        roc_auc = auc(fpr, tpr)
        
        interp_tpr = np.interp(mean_fpr, fpr, tpr)
        interp_tpr[0] = 0.0
        tprs.append(interp_tpr)
        aucs.append(roc_auc)

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    np.mean(MCCs)
    np.std(MCCs)
    stacked_matrices = np.stack(cms, axis=0)
    mean_cms = np.mean(stacked_matrices, axis=0)
    mean_cms = np.round(mean_cms).astype(
        int,
    )  
    print(f"Mean MCC: {np.mean(MCCs):.3f} ± {np.std(MCCs):.3f}")
    print(f"Mean balanced accuracy: {np.mean(baccs):.3f} ± {np.std(baccs):.3f}")
    print(f"Mean recall: {np.mean(recalls):.3f} ± {np.std(recalls):.3f}")
    print(f"Mean precision: {np.mean(precisions):.3f} ± {np.std(precisions):.3f}")
    print(f"Mean AUC: {mean_auc:.3f} ± {std_auc:.3f}")
    
    ax.plot(
        [0, 1],
        [0, 1],
        linestyle="--",
        lw=2,
        color="r",
        label="Chance",
        alpha=0.5,
    )
    ax.plot(
        mean_fpr,
        mean_tpr,
        color="navy",
        label=rf"Mean ROC (AUC = {mean_auc:0.2f} $\pm$ {std_auc:0.2f})",
        lw=2,
        alpha=1,
    )
    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)

    ax.fill_between(
        mean_fpr,
        tprs_lower,
        tprs_upper,
        color="grey",
        alpha=0.2,
    )
    ax.set(
        xlim=[-0.05, 1.05],
        ylim=[-0.05, 1.05],
        title="ROC Curve",
    )
    ax.legend(loc="lower right")
    plt.savefig('/home/hzhang/HuanniZ/Data/Submissiion_supplement_figuersAndfiles/Figure4A.roc_curve_M2FPs.pdf', format='pdf', bbox_inches='tight')
    disp = ConfusionMatrixDisplay(
        confusion_matrix=mean_cms,
        display_labels=['benign', 'toxic']
    )
    disp.plot(
        cmap=plt.cm.Blues,
        # xticks_rotation="vertical"
    )
    plt.savefig('/home/hzhang/HuanniZ/Data/Submissiion_supplement_figuersAndfiles/Figure4A.confusion_matrix_M2FPs.pdf', format='pdf', bbox_inches='tight')

In [31]:
MLP_cv(np.array(X),y,opt_num_feature=750,threshold=0.12)

## model based on M2FPs+PBFPs

In [32]:
M2FPs_PBFPs =  morgan2_cols + targets_list
len(M2FPs_PBFPs)

In [33]:
X = Training_data[M2FPs_PBFPs]
y = Training_data.Mark

In [ ]:
opt_threshold_ap, opt_threshold_mean = MLP_decision_threshold_optimization(
    X,
    y,
    opt_num_feature=850)

In [34]:
MLP_cv(np.array(X),y, opt_num_feature=850, threshold=0.15)

# Simulation tests

In [ ]:
X = Training_data[M2FPs_PBFPs]
selector = SelectKBest(f_classif, k=850)
X_new = selector.fit_transform(X, y)

In [ ]:
k_fold_mccs = []
for iter in tqdm(range(1, 10001)): 
    k_fold_mccs.append(MLP_simulation_test(X_new,y))

In [ ]:
plt.hist(k_fold_mccs, bins=100);
plt.xlabel('MCC', fontsize = 10) 
plt.ylabel('Count',fontsize = 10) 
plt.title('Simulation test')
plt.axvline(x = 0.46, color = 'b',lw=2,label = 'MCC of MLP based on\nM2FPs+PBFPs = 0.46')
plt.axvline(x = 0.44, color = 'r',lw=2,label = 'MCC of MLP based on\nM2FP = 0.44')
plt.axvline(x = 0.09, color = 'g',lw=2,label = 'MCC of MLP based on\nGIN = 0.09')
plt.axvline(x = 0.17, color = 'm',lw=2,label = 'MCC of MLP based on\nChemBERTa = 0.17')
plt.legend(loc='upper right')

# Hyperparameter optimization

MLP

In [ ]:
X = Training_data[M2FPs_PBFPs]
y = Training_data.Mark
selector = SelectKBest(f_classif, k=850)
X_new = selector.fit_transform(X, y)

param_grid = {
    'hidden_layer_sizes':[(10,),(20,),(30,),(40,),(50,),(60,),(70,),(80,),(90,),(100,)],
    'learning_rate_init':[0.0001,0.001,0.01,0.1],
    'random_state':[42],
    'max_iter':[200, 400, 600, 800]
    }
MLP = MLPClassifier()
grid_search = GridSearchCV(
    estimator = MLP, 
    param_grid = param_grid, 
    scoring='roc_auc',
    cv = 10)
grid_search.fit(X_new, y)
grid_search.best_params_

# Feature selecttion (K value optimization)

Example of model based on M2FPs+PBFPs features

initial

In [ ]:
X = Training_data[M2FPs_PBFPs]
y = Training_data.Mark
ANOVA_K_list = ['100','500','1000','1500','2000']
feature_selection(X, y, M2FPs_PBFPs,'M2FPs + PBFPs', ANOVA_K_list) 

extended

In [ ]:

ANOVA_K_list = ['500','550','600','650','700','750','800','850','900','950','1000']
feature_selection(X, y, M2FPs_PBFPs,'M2FPs + PBFPs', ANOVA_K_list) 